# 04. Variant Annotation (SnpEff Custom Build)

This notebook annotates our VCF files to predict biological impact.

Our reference genome (`GCF_022832775.1`) is not in the public SnpEff database, so we must build our own custom database.

**Clean Workflow:**
1.  **Download Source Files:** Download the **GenBank (GBFF)** and **FASTA (Genome)** files from NCBI. SnpEff builds best from GenBank files.
2.  **Organize Files:** Create the required SnpEff directory structure (`snpEff/data/[genome_name]/`) and rename the files to `genes.gbk` and `sequences.fa`.
3.  **Update Config:** Add our new custom genome to the `snpEff.config` file.
4.  **Run `snpEff build`:** Compile the custom database using the `-genbank` flag.

In [ ]:
import os
import subprocess

# --- 1. Download Source Files (FASTA + GenBank) ---
print("--- Downloading FASTA (genome) and GenBank (gbff) files... ---")
# We use 'gbff' (the correct flag) and 'genome'
!datasets download genome accession GCF_022832775.1 --filename ../references/genome_data.zip --include genome,gbff

# --- 2. Unzip & Organize Files ---
print("\n--- Unzipping and organizing files for SnpEff... ---")
!unzip -o ../references/genome_data.zip -d ../references/

# Define paths
genome_name = "GCF_022832775.1"
base_data_path = f"../references/ncbi_dataset/data/{genome_name}"
snpeff_data_path = f"../references/snpEff/data/{genome_name}"

# Create the target SnpEff directory
# The -p flag will create all parent directories (snpEff, data) that are missing
!mkdir -p {snpeff_data_path}

# Move and rename files using *exact* filenames (no conflicting wildcards)
!mv {base_data_path}/GCF_022832775.1_ASM2283277v1_genomic.fna {snpeff_data_path}/sequences.fa
!mv {base_data_path}/genomic.gbff {snpeff_data_path}/genes.gbk

# --- 3. Clean up ---
!rm ../references/genome_data.zip
!rm -rf ../references/ncbi_dataset
!rm ../references/README.md

# --- 4. Verify the SnpEff directory structure ---
print("\n--- Verification: SnpEff custom data directory (must have 2 files) ---")
!ls -lh {snpeff_data_path}

## 2. Build Custom SnpEff Database

Now that our clean source files (`genes.gbk` and `sequences.fa`) are perfectly organized in our custom data directory, we can proceed with the build.

**Workflow:**
1.  **Update Config:** Add our custom genome to the `snpEff.config` file (this will be skipped if already present).
2.  **Run `snpEff build`:** Compile the custom database using the `-genbank` flag and pointing to our custom `dataDir`.

In [ ]:
import os
import subprocess

# 1. Find the snpEff config file path
snpeff_path_bytes = subprocess.check_output(['which', 'snpEff'])
snpeff_path = snpeff_path_bytes.decode('utf-8').strip()
snpeff_dir = os.path.dirname(snpeff_path)
share_dir_base = os.path.abspath(os.path.join(snpeff_dir, '..', 'share'))
snpeff_share_dir = ""
for item in os.listdir(share_dir_base):
    if item.startswith("snpeff-"):
        snpeff_share_dir = os.path.join(share_dir_base, item)
        break
if not snpeff_share_dir:
     raise FileNotFoundError("Could not find snpEff share directory")
config_path = os.path.join(snpeff_share_dir, 'snpEff.config')
print(f"--- Found snpEff config file at: {config_path} ---")

# 2. Define our custom genome entry
genome_name = "GCF_022832775.1"
genome_description = "Staphylococcus aureus ATCC 29213 GCF_022832775.1"
custom_entry = f"\n# Custom S. aureus ATCC 29213 Genome\n{genome_name}.genome : {genome_description}\n"

# 3. Check/Add entry to config
entry_exists = False
try:
    with open(config_path, 'r') as f:
        if genome_name in f.read():
            entry_exists = True
            print(f"--- Genome entry for {genome_name} already exists in config. Skipping append. ---")
except FileNotFoundError:
    print(f"--- Config file not found at {config_path}, cannot check existence. Proceeding with append. ---")
if not entry_exists:
    print(f"--- Appending custom entry to: {config_path} ---")
    add_command = f"echo -e '{custom_entry}' | sudo tee -a {config_path}"
    process = subprocess.run(add_command, shell=True, capture_output=True, text=True)
    if process.returncode != 0:
        print(f"--- ERROR appending to config file: {process.stderr} ---")
    else:
        print("--- Entry potentially appended successfully. ---")

# 4. Use Absolute Path for dataDir
absolute_data_dir = os.path.abspath("../references/snpEff/data")
print(f"--- Using Absolute Path for dataDir: {absolute_data_dir} ---")

# 5. Build the SnpEff database using the -genbank flag
print(f"\n--- Building SnpEff database for {genome_name} using -genbank flag... ---")
build_command = f"snpEff build -dataDir {absolute_data_dir} -genbank {genome_name} -v"
!{build_command}

# 6. Verify the build
print(f"\n--- Verifying database build (looking for {genome_name}): ---")
!snpEff databases | grep "{genome_name}"

## 3. Run SnpEff Annotation

The custom database `GCF_022832775.1` is now successfully built.

We can now *finally* loop through our 7 VCF files from GATK and run `snpEff` to annotate them.

**Workflow:**
1.  **Input:** Our 7 `.vcf.gz` files from `../results/vcf_files/`.
2.  **Process:** `snpEff` will use our new custom database (`GCF_022832775.1`) to check every variant and determine its genomic location (e.g., in a gene, intergenic) and its effect (e.g., `missense_variant`, `synonymous_variant`, `frameshift`).
3.  **Output:**
    * **Annotated VCFs:** 7 new `.ann.vcf.gz` files (annotated VCFs) in `../results/annotated_vcf/`.
    * **HTML Reports:** 7 new `.html` summary reports in `../results/snpeff_reports/`.

In [ ]:
%%bash
# Tell Jupyter to run this entire cell as a bash script

# 1. Define our custom genome name and data directory path
GENOME_NAME="GCF_022832775.1"
ABSOLUTE_DATA_DIR=$(realpath ../references/snpEff/data)

echo "--- Using Genome: $GENOME_NAME ---"
echo "--- Using DataDir: $ABSOLUTE_DATA_DIR ---"

# 2. Create output directories
mkdir -p ../results/annotated_vcf/
mkdir -p ../results/snpeff_reports/

# 3. Loop through all VCF files from GATK
for vcf_path in ../results/vcf_files/*.vcf.gz; do
    
    # --- Construct filenames ---
    vcf_filename=$(basename "$vcf_path")
    sample_id=$(echo "$vcf_filename" | cut -d'.' -f1) # Cuts at the first '.'
    
    echo "--- Starting SnpEff Annotation for sample: $sample_id ---"
    
    # Define paths for the new annotated VCF and the HTML report
    output_vcf="../results/annotated_vcf/${sample_id}.ann.vcf.gz"
    output_html="../results/snpeff_reports/${sample_id}.snpEff_summary.html"
    
    # --- Run snpEff ---
    # -v: Verbose (to see progress)
    # -dataDir: We *must* tell SnpEff where our custom database is
    # -s: Create an HTML summary report
    # $GENOME_NAME: The name of the database to use
    
    snpEff ann \
        -v \
        -dataDir $ABSOLUTE_DATA_DIR \
        -s $output_html \
        $GENOME_NAME \
        $vcf_path \
        | gzip > $output_vcf # Pipe (|) and compress the output VCF
    
    echo "--- Finished SnpEff for sample: $sample_id ---"
done

echo "--- All SnpEff annotations are complete. ---"

In [ ]:
# Verify the new Annotated VCFs and HTML reports
print("--- Verification: SnpEff Annotated VCFs (.ann.vcf.gz) ---")
!ls -lh ../results/annotated_vcf/

print("\n--- Verification: SnpEff HTML Reports (.html) ---")
!ls -lh ../results/snpeff_reports/

### MAJOR STEP 3: Annotation Summary and Aggregation Decision

We have successfully completed the variant annotation pipeline for all 7 samples. 

**Output:** 7 annotated VCF files (`results/annotated_vcf/*.ann.vcf.gz`) and 7 detailed SnpEff reports (`results/snpeff_reports/*.genes.txt`).

**Decision on Aggregation (MultiQC):**
During the final aggregation attempt, we encountered a fundamental incompatibility issue between the version of MultiQC installed and the output format of the SnpEff gene reports. Extensive debugging confirmed that forcing the aggregation would require complex file manipulation that compromises the clean pipeline principle.

**Conclusion:**
To maintain **data integrity** and ensure the final **comparative analysis** (Control vs. DAP/VAN) is robust and transparent, we will abandon the MultiQC aggregation step and proceed with a dedicated, programmatic **downstream analysis using Python (Pandas)** in the next notebook. The raw report files are now ready for interpretation.